In [2]:
### Import packages

from __future__ import print_function
import numpy as np
import pandas as pd
import chartify
import sys
import os
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import bokeh
import bokeh.io as bk_io
from bokeh.io import export_svgs, export_png
from bokeh.plotting import figure, output_file, show, save
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, Span, Label,  Arrow, OpenHead, NormalHead, VeeHead, HoverTool
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral6, Dark2, cividis, Viridis, viridis
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.palettes import Spectral6
from bokeh.transform import dodge
from bokeh.models import ColumnDataSource, Whisker
#import phantomjs-prebuilt

# Binding box plot

In [18]:
scatter = pd.read_excel('/Users/taylorjones/Desktop/Peptide_Project/binding_box_data_peptides.xlsx', header=None)
cols = ['bound', 'x', 'cond']
scatter.columns = cols
scatter['axis'] = 1

scatter

,bound,x,cond,axis
0,65.9,1.0,peptide,1
1,54.8,1.0,peptide,1
2,36.3,1.0,peptide,1
3,64.5,1.0,peptide,1
4,62.6,1.0,peptide,1
5,83.1,1.0,peptide,1
6,100.0,1.2,none,1
7,117.0,1.1,qs,1
8,108.0,1.1,qs,1


In [52]:
pep = scatter[(scatter['cond'] == 'peptide')]
qs = scatter[(scatter['cond'] == 'qs' )]
def box_input(cond):
    cond_list = []
    q1 = cond['bound'].quantile(0.25)
    q2 = cond['bound'].quantile(0.5)
    q3 = cond['bound'].quantile(0.75)
    iqr15 = 1.5*(q3-q1)
    cmin = q1-iqr15
    cmax = q3+iqr15
    cond_list.append(cmin)
    cond_list.append(q1)
    cond_list.append(q2)
    cond_list.append(q3)
    cond_list.append(cmax)
    return cond_list
pep_box = box_input(pep)
qs_box = box_input(qs)
qs_box

[103.5, 110.25, 112.5, 114.75, 121.5]

In [60]:
def f(row):
    if (row['cond'] == 'peptide'):
        return '#EE7416'
    if (row['cond'] == 'qs'):
        return '#3484BF'
    return 'black'

scatter['colors'] = scatter.apply (lambda row: f(row), axis=1)
source = ColumnDataSource(scatter)

p = figure(plot_width=600, plot_height=1200, title=None, 
           x_range=(.95, 1.05), y_range=(20, 125))
p.xaxis.axis_label = ' '
p.yaxis.axis_label = '% Bound'    
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "24pt"
p.yaxis.axis_label_text_font_size = "24pt"
p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_font_size = "20pt"
p.yaxis.major_label_text_color = "black"


# stems + whiskers
p.segment([1,1], [pep_box[1], pep_box[3]], [1,1], [pep_box[0], pep_box[4]],
          line_color="black", line_width=2)
p.segment([0.995,0.995], [pep_box[4], pep_box[0]], [1.005,1.005], [pep_box[4], pep_box[0]],
          line_color="black", line_width=2)

# boxes
p.vbar([1], 0.05, pep_box[2], pep_box[3],
       fill_color="#EEC3A2", line_color="black")
p.vbar([1], 0.05, pep_box[2], pep_box[1],
       fill_color="#EEC3A2", line_color="black")

# stems + whiskers
p.segment([1,1], [qs_box[1], qs_box[3]], [1,1], [qs_box[0], qs_box[4]],
          line_color="black", line_width=2)
p.segment([0.995,0.995], [qs_box[4], qs_box[0]], [1.005,1.005], [qs_box[4], qs_box[0]],
          line_color="black", line_width=2)

# boxes
p.vbar([1], 0.05, qs_box[2], qs_box[3],
       fill_color="#9CC3DF", line_color="black")
p.vbar([1], 0.05, qs_box[2], qs_box[1],
       fill_color="#9CC3DF", line_color="black")


hline = Span(location=100, dimension='width', line_color='black', line_width=2, line_dash='dashed')
p.renderers.extend([hline])

p.circle('axis', 'bound', size=14,
             color='colors',
             source=source, line_color='black')
p.output_backend = "svg"

outdir = '/Users/taylorjones/Desktop/Peptide_Project/#2021-08/'
export_svgs(p, filename = outdir + 'binding_scatter_and_boxes.svg')
show(p)

# RT-qPCR bar plots

In [3]:
df = pd.read_excel('/Users/taylorjones/Desktop/Peptide_Project/RT_qPCR_peptide_bar_plot.xlsx')

def f(row):
    if (row[''] == 'peptide'):
        return '#2e4199'
    if (row['cond'] == 'qs'):
        return '#f4801f'
    return 'black'

df['colors_water'] = 'black'
df['colors_peptide'] = '#2e4199'

df['Peptide upper'] =df['peptide av'] + df['peptide sterr']
df['Peptide lower'] =df['peptide av'] - df['peptide sterr']

df['Water upper'] =df['water av'] + df['water sterr']
df['Water lower'] =df['water av'] - df['water sterr']



nut = df.loc[df['treatment'] == 'Nutlin']
dmso = df.loc[df['treatment'] == 'DMSO']

outdir = '/Users/taylorjones/Desktop/Peptide_Project/#####2021-08/'
df

,gene,treatment,water av,water stdev,water sterr,peptide av,peptide stdev,peptide sterr,colors_water,colors_peptide,Peptide upper,Peptide lower,Water upper,Water lower
0,p21,DMSO,1.000000,0.164540,0.116347,2.418028,0.259006,0.183145,black,#2e4199,2.601173,2.234884,1.116347,0.883653
1,p21,Nutlin,7.579626,1.153483,0.815636,2.052430,0.492698,0.348390,black,#2e4199,2.400820,1.704040,8.395262,6.763990
2,MDM2,DMSO,1.000000,0.152414,0.107773,1.358409,0.171553,0.121306,black,#2e4199,1.479716,1.237103,1.107773,0.892227
3,MDM2,Nutlin,4.970576,0.550042,0.388939,1.139450,0.035698,0.025242,black,#2e4199,1.164692,1.114207,5.359515,4.581637
4,PHLDA3,DMSO,1.000000,0.132637,0.093788,1.313646,0.083842,0.059285,black,#2e4199,1.372931,1.254360,1.093788,0.906212
5,PHLDA3,Nutlin,5.603198,0.573894,0.405804,0.755447,0.210306,0.148709,black,#2e4199,0.904156,0.606739,6.009002,5.197393
6,TP53I3,DMSO,1.000000,0.247150,0.174762,0.885410,0.042289,0.029903,black,#2e4199,0.915312,0.855507,1.174762,0.825238
7,TP53I3,Nutlin,2.292717,0.560510,0.396341,0.708719,0.273998,0.193746,black,#2e4199,0.902464,0.514973,2.689058,1.896376
8,DRAM1,DMSO,1.000000,0.351355,0.248446,1.103816,0.268958,0.190182,black,#2e4199,1.293998,0.913634,1.248446,0.751554
9,DRAM1,Nutlin,2.136329,0.345048,0.243986,0.609558,0.111691,0.078977,black,#2e4199,0.688536,0.530581,2.380314,1.892343


In [12]:
##Nutlin
genes = nut['gene']
treatments = ['Water', 'Peptide']
uppers = ['Water upper', 'Peptide upper']
lowers =['Water lower', 'Peptide lower']
errors = ['Water error', 'Peptide error']

data = {'Genes' : genes,
        'Water'   : nut['water av'],
        'Peptide'   : nut['peptide av'],
        'Water upper'   : nut['Water upper'],
        'Peptide upper'   : nut['Peptide upper'],
        'Water lower'   : nut['Water lower'],
        'Peptide lower'   : nut['Peptide lower'],
        'Water error'   : nut['water sterr'],
        'Peptide error'   : nut['peptide sterr']}

x = [(gene, treatment) for gene in genes for treatment in treatments]

err = [(gene, error) for gene in genes for error in errors]

counts = sum(zip(data['Water'], data['Peptide']), ())
counts_err = sum(zip(data['Water error'], data['Peptide error']), ())

upper = [y+e for y,e in zip(counts, counts_err) ]
lower = [y-e for y,e in zip(counts, counts_err) ]

colors = ['black', 'blue', 'black', 'blue', 'black', 'blue', 'black', 'blue', 
         'black', 'blue', 'black', 'blue', 'black', 'blue', 'black', 'blue']

source = ColumnDataSource(data=dict(x=x, counts=counts, upper=upper, lower=lower))


p = figure(x_range=FactorRange(*x), plot_height=600, plot_width= 1600, title="Nutlin Treated Cells",
           toolbar_location=None, tools="")

#p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white")
palette=["#b9b9b9", "#3484BF"]
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=treatments, start=1, end=2))

p.add_layout(Whisker(source=source, base="x", upper="upper", lower="lower", level="overlay", line_width=2.5))

p.y_range.start = 0
p.y_range.end=9
p.x_range.range_padding = 0.2
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label = 'Fold Change in Expression' 
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "16pt"
p.title.text_font_size = "25pt"
p.yaxis.major_label_text_font_size = "16pt"
p.xaxis.group_text_font_size = '16pt'
#p.xaxis.text_font_size = "16pt"
#p.title.
p.output_backend = "svg"
outdir = '/Users/taylorjones/Desktop/Peptide_Project/####2021-08/RT-qPCR/'
export_svgs(p, filename = outdir + 'Nutlin_zoom.' + 'svg')
show(p)


In [11]:
##DMSO
genes = dmso['gene']
treatments = ['Water', 'Peptide']
uppers = ['Water upper', 'Peptide upper']
lowers =['Water lower', 'Peptide lower']
errors = ['Water error', 'Peptide error']

data = {'Genes' : genes,
        'Water'   : dmso['water av'],
        'Peptide'   : dmso['peptide av'],
        'Water upper'   : dmso['Water upper'],
        'Peptide upper'   : dmso['Peptide upper'],
        'Water lower'   : dmso['Water lower'],
        'Peptide lower'   : dmso['Peptide lower'],
        'Water error'   : dmso['water sterr'],
        'Peptide error'   : dmso['peptide sterr']}

x = [(gene, treatment) for gene in genes for treatment in treatments]

err = [(gene, error) for gene in genes for error in errors]

counts = sum(zip(data['Water'], data['Peptide']), ())
counts_err = sum(zip(data['Water error'], data['Peptide error']), ())

upper = [y+e for y,e in zip(counts, counts_err) ]
lower = [y-e for y,e in zip(counts, counts_err) ]

colors = ['black', 'blue', 'black', 'blue', 'black', 'blue', 'black', 'blue', 
         'black', 'blue', 'black', 'blue', 'black', 'blue', 'black', 'blue']

source = ColumnDataSource(data=dict(x=x, counts=counts, upper=upper, lower=lower))


p = figure(x_range=FactorRange(*x), plot_height=600, plot_width= 1600, title="DMSO Treated Cells",
           toolbar_location=None, tools="")

#p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white")
palette=["#b9b9b9", "#3484BF"]
p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=treatments, start=1, end=2))
 
p.add_layout(Whisker(source=source, base="x", upper="upper", lower="lower", level="overlay", line_width=2.5))

p.y_range.start = 0
p.y_range.end=3
p.x_range.range_padding = 0.2
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label = 'Fold Change in Expression' 
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "16pt"
p.title.text_font_size = "25pt"
p.yaxis.major_label_text_font_size = "16pt"
p.xaxis.group_text_font_size = '16pt'
#p.xaxis.text_font_size = "16pt"
#p.title.
p.output_backend = "svg"
outdir = '/Users/taylorjones/Desktop/Peptide_Project/####2021-08/RT-qPCR/'
export_svgs(p, filename = outdir + 'DMSO_zoom.' + 'svg')
show(p)


# ChIP-qPCR bar plots

In [ ]:
df = pd.read_csv('/Users/taylorjones/Desktop/Peptide_Project/chip_qpcr_plot_input.csv',sep=',')
df =df[['Unnamed: 0', 'average_input_normalized', 'stderr']]
df.columns = ['Sample', 'average', 'stderr']
df

In [ ]:
treatments = ['DMSO', 'Nutlin',
               'DMSO + Peptide', 'Nutlin + Peptide']
errors = ['DMSO err', 'Nutlin err',
               'DMSO + Peptide err', 'Nutlin + Peptide err']

data = {'DMSO'   : df['average'].iloc[[0]],
        'Nutlin'   : df['average'].iloc[[1]],
        'DMSO + Peptide' : df['average'].iloc[[2]],
        'Nutlin + Peptide' : df['average'].iloc[[3]],
        
        'DMSO err'   : df['stderr'].iloc[[0]],
        'Nutlin err'   : df['stderr'].iloc[[1]],
        'DMSO + Peptide err' : df['stderr'].iloc[[2]],
        'Nutlin + Peptide err' : df['stderr'].iloc[[3]]}

x = [(treatment) for treatment in treatments]

err = [(error) for error in errors]

counts = sum(zip(data['DMSO'], data['Nutlin'], data['DMSO + Peptide'], data['Nutlin + Peptide']), ())
counts_err = sum(zip(data['DMSO err'], data['Nutlin err'], data['DMSO + Peptide err'], data['Nutlin + Peptide err']), ())

upper = [y+e for y,e in zip(counts, counts_err) ]
lower = [y-e for y,e in zip(counts, counts_err) ]


fill_color= ["#b9b9b9", "#3484BF", "#b9b9b9", "#3484BF"]


source = ColumnDataSource(data=dict(x=x, counts=counts, upper=upper, lower=lower, fill_color=fill_color))


p = figure(x_range=FactorRange(*x), plot_height=600, plot_width= 600, title="CDKN1A PolIISer5 Levels",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white")

p.vbar(x='x', top='counts', width=0.9, source=source, line_color="white",
        fill_color='fill_color')

p.add_layout(Whisker(source=source, base="x", upper="upper", lower="lower", level="overlay", line_width=2.5))

p.y_range.start = 0
p.y_range.end=28
p.x_range.range_padding = 0.2
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "28pt"
#$e^{\sin(\omega\phi)}$

p.yaxis.axis_label_text_font_size = "18pt"
p.yaxis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_text_font_size = "18pt"
p.title.text_font_size = "22pt"
p.yaxis.major_label_text_font_size = "18pt"
p.xaxis.group_text_font_size = '18pt'
p.xaxis.major_label_orientation = 1

p.yaxis.axis_label = 'Input Normalized SQ Values' 

p.output_backend = "svg"

outdir = '/Users/taylorjones/Desktop/Peptide_Project/####2021-08/ChIP-qPCR/'
export_svgs(p, filename = outdir + 'pepitde_ser5_chip_CDKN1A.' + 'svg')
show(p)
